In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

import numpy as np
from keras.optimizers import SGD
import random
import pandas as pd
import re, string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Casio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Casio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv(r"C:\Users\Casio\miniconda3\envs\gp_bank_2\data\fqa.csv")
df                

,Unnamed: 0,Question,Topic,Answer
0,0,Какие банковские продукты предлагает банк?,Карта,"Наш банк предлагает оформление виртуальных, пр..."
1,1,Какие премиальные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих преми...
2,2,Какие виртуальные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих вирту...
3,3,Какие расчетные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих расче...
4,4,Предалагаете ли вы карточки для нерезидентов?,Карта,Наш банк предлагает оформление карточек для не...
...,...,...,...,...
194,194,Банкоматы в Заводском районе Минска?,Адрес,"г. Минск улица Притыцкого 79, г. Минск улица ..."
195,195,Банкоматы в Ленинском районе Минска?,Адрес,"г. Минск улица Притыцкого 79, г. Минск улица ..."
196,196,Банкоматы в Октябрьском районе Минска?,Адрес,"г. Минск улица Притыцкого 79, г. Минск улица ..."
197,197,Банкоматы в Московском районе Минска?,Адрес,"г. Минск улица Притыцкого 79, г. Минск улица ..."


In [7]:
def preprocess(text):
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('russian')]
    return ' '.join(a)

wl = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) 
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] 
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [8]:
df['clean_question'] = df['Question'].apply(lambda x: finalpreprocess(x))
df.head()

,Unnamed: 0,Question,Topic,Answer,clean_question
0,0,Какие банковские продукты предлагает банк?,Карта,"Наш банк предлагает оформление виртуальных, пр...",какие банковские продукты предлагает банк
1,1,Какие премиальные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих преми...,какие премиальные карты предлагает банк
2,2,Какие виртуальные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих вирту...,какие виртуальные карты предлагает банк
3,3,Какие расчетные карты предлагает банк?,Карта,Наш банк предлагает оформление следующих расче...,какие расчетные карты предлагает банк
4,4,Предалагаете ли вы карточки для нерезидентов?,Карта,Наш банк предлагает оформление карточек для не...,предалагаете карточки нерезидентов


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

cv_quest = cv.fit_transform(df['clean_question']).toarray()

In [8]:
X = cv_quest
y = df["Answer"].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state = 42)

In [9]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
result_bayes = nb.fit(x_train, y_train)
nb.score(x_test,y_test)

0.35714285714285715

In [10]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
result_logreg = lg.fit(x_train, y_train)
lg.score(x_test,y_test)

0.37142857142857144

In [11]:
from sklearn import svm
svm = svm.SVC()
result_svm = svm.fit(x_train, y_train)
svm.score(x_test, y_test)

0.32857142857142857

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

adamodelClf = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, random_state=42)

adamodelclf_fit = adamodelClf.fit(x_train, y_train)
adamodelClf.score(x_test, y_test)

0.3

In [13]:
from sklearn.ensemble import GradientBoostingClassifier


modelClf = GradientBoostingClassifier(max_depth=2, n_estimators=100,random_state=42, learning_rate=0.2)

modelClf.fit(x_train, y_train)
modelClf.score(x_test, y_test)

0.4

In [14]:
X_new = 'Какие карточки вы выпускаете?'
clean_X_new = finalpreprocess(X_new)
cv_new = cv.transform([clean_X_new]).toarray()
Y_new = modelClf.predict(cv_new)[0]

In [15]:
Y_new

'БЕЛКАРТ-ПРЕМИУМ для нерезидентов предлагает следующие условия: Валюта текущего счета: BYN, Срок действия карты: до 5 лет, Выпуск карточки осуществляется в офисах ОАО «Белгазпромбанка», Возможность выпуска дополнительных карточек'

In [16]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((129, 182), (70, 182), (129,), (70,))

In [109]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

tfidf_params = {
    'analyzer': 'word',
    'max_df': 0.99,
    'binary': True,
    'max_features': 1000}

In [120]:
dist_params = {
    'n_neighbors': 1,
    'metric': 'cosine'}

In [121]:
vectorizer = TfidfVectorizer(**tfidf_params)
dist = NearestNeighbors(**dist_params)

In [122]:
X = vectorizer.fit_transform(df['Question'])
dist.fit(X, df['Answer'])

NearestNeighbors(metric='cosine', n_neighbors=1)

In [123]:
def transform_text(vectorizer, text):
    print('Text:', text)
    vector = vectorizer.transform([text])
    vector = vectorizer.inverse_transform(vector)
    print('Vect:', vector)

In [124]:
X_new = 'какие карты для нерезидентов есть в вашем банке?'
X = vectorizer.transform([X_new])
dist_num = dist.kneighbors(X, return_distance=False)[0][0]
selected = df.iloc[dist_num]['Answer']

In [125]:
y_pred = transform_text(vectorizer, selected)
y_pred

Text: Наш банк оформляет карточки платежной системы Mastercard: Виртуальная карточка Mastercard Gold, Расчетная карточка Mastercard Standard, Премиальная карточка Mastercard Black Edition, Премиальная карточка Mastercard World, Премиальная карточка Masterсard Gold, Mastercard World Elite, «Карта учредителя» Mastercard Gold, Masterсard Gold «Профи», Masterсard Gold для нерезидентов
Vect: [array(['учредителя', 'профи', 'платежной', 'нерезидентов', 'карточки',
       'карта', 'для', 'банк', 'world', 'standard', 'masterсard',
       'mastercard', 'gold', 'elite', 'edition', 'black'], dtype='<U15')]


In [4]:
class TfidfModel():
    
    def __init__(self, max_df=0.95, random_state = 42):
        self.vectorizer = TfidfVectorizer(max_df=max_df)
        self.classifier = GradientBoostingClassifier(max_depth=2, n_estimators=1000,random_state=42, learning_rate=0.1)
        
        
    def fit (self, X, y):
        x_tfidf = self.vectorizer.fit_transform(X)
        self.classifier.fit(x_tfidf, y)
        
    def predict (self, X):
        x_tfidf = self.vectorizer.transform(X)
        return self.classifier.predict(x_tfidf)
    
    def score (self, X, y):
        y_pred = self.predict(X)
        return accuracy_score (y, y_pred)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_question'], df['Answer'], test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

model = TfidfModel()
model.fit(X_train,y_train)

In [9]:
df_pred = pd.DataFrame(X_test)
df_pred['predict_answer'] = model.predict(df_pred['clean_question'])
df_pred.head(5)

,Question,predict_answer
82,Поддерживает ли карта UnionPay Platinum бескон...,Валюта текущего счета премиальной карточки Vis...
15,Какой срок действия по карте Cashalot?,Срок действия премиальной карточки Visa Gold д...
111,Какие условия кредитования предлагаются пользо...,Индивидуальные условия кредитования в банке
177,Есть ли карты с технологией бесконтактных плат...,Есть возможность подключения карточки к платеж...
76,Можно ли подключить Mastercard Black Edition ...,Есть возможность подключения карточки к платеж...


In [10]:
accuracy = model.score(X_test, y_test)
accuracy

0.35

In [11]:
class TfidfModel_knn():
    
    def __init__(self, max_df=0.99, random_state = 42):
        self.vectorizer = TfidfVectorizer(max_df=max_df)
        self.classifier = MultiOutputClassifier(KNeighborsClassifier (n_neighbors = 1,metric = 'cosine'))
        
        
    def fit (self, X, y):
        x_tfidf = self.vectorizer.fit_transform(X)
        self.classifier.fit(x_tfidf, y)
        
    def predict (self, X):
        x_tfidf = self.vectorizer.transform(X)
        indices = self.classifier.predict(x_tfidf)
        return indices
    
    def score (self, X, y):
        y_pred = self.predict(X)
        return accuracy_score (y, y_pred)

In [12]:
model_knn = TfidfModel_knn()
y_train = y_train.values.reshape(-1, 1)
model_knn.fit(X_train,y_train)

In [13]:
df_pred_knn = pd.DataFrame(X_test)
df_pred_knn['predict_answer'] = model_knn.predict(df_pred_knn['clean_question']).flatten()
df_pred_knn.head(5)

,Question,predict_answer
82,Поддерживает ли карта UnionPay Platinum бескон...,Валюта текущего счета премиальной карточки Uni...
15,Какой срок действия по карте Cashalot?,Срок действия виртуальной карты Cashalot до 5 лет
111,Какие условия кредитования предлагаются пользо...,Индивидуальные условия кредитования в банке
177,Есть ли карты с технологией бесконтактных плат...,Включена программа «Бесценные города» с доступ...
76,Можно ли подключить Mastercard Black Edition ...,Есть возможность подключения карточки к платеж...


In [14]:
accuracy = model_new.score(X_test, y_test)
accuracy

0.35

In [14]:
import navec
import numpy as np
import pandas as pd
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the Navec model
#navec = navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [98]:
import torchtext.vocab

class TfidfModel_emdeding():
    def __init__(self, max_df=0.99, random_state=42, embedding_path=None):
        self.max_df = max_df
        self.random_state = random_state
        self.embedding_path = "cc.ru.300.vec"
        self.vectorizer = TfidfVectorizer(max_df=max_df, vocabulary=self.load_embedding())
        self.classifier = MultiOutputClassifier(KNeighborsClassifier(n_neighbors=5, metric='cosine'))

    def load_embedding(self):
        if self.embedding_path is None:
            return None

        embedding = torchtext.vocab.Vectors(self.embedding_path)

        
        vocab = embedding.stoi

        return vocab

    def fit(self, X, y):
        x_tfidf = self.vectorizer.fit_transform(X)
        self.classifier.fit(x_tfidf, y)

    def predict(self, X):
        x_tfidf = self.vectorizer.transform(X)
        indices = self.classifier.predict(x_tfidf)
        return indices

    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)

In [99]:
model_em = TfidfModel_embedding()
model_em.fit(X_train,y_train)

C:\Users\Casio\miniconda3\envs\project\Lib\site-packages\sklearn\feature_extraction\text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [100]:
df_pred_em = pd.DataFrame(X_test)
df_pred_em['predict_answer'] = model_em.predict(df_pred_em['clean_question']).flatten()
df_pred_em.head(5)

,clean_question,predict_answer
82,поддерживает карта unionpay platinum бесконтак...,Валюта текущего счета премиальной карточки Uni...
15,срок действия карте cashalot,Срок действия премиальной карточки Masterсard ...
111,какие условия кредитования предлагаются пользо...,Mastercard World Elite предлагает следующие ус...
177,карты технологией бесконтактных платежей,Наш банк оформляет карточки платежной системы ...
76,подключить mastercard black edition samsung pay,Есть возможность подключения карточки к платеж...


In [101]:
accuracy = model_em.score(X_test, y_test)
accuracy

0.35